In [1]:
# Required imports.
from py import calc_ubi
from py.loss_functions import loss_metrics
from py.optimize import optimize

import pandas as pd

In [2]:
AGE_CATEGORIES = ["senior", "child"]
DIS_CATEGORIES = ["dis_base"]
REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]
categories = AGE_CATEGORIES + DIS_CATEGORIES + REGIONS

# Define bounds
AGE_BOUNDS = (40, 240)  # Child, working-age adult, senior.
DIS_BOUNDS = (0, 200)  # Base, severe, and enhanced disability UBI supplements.
GEO_BOUNDS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
bounds = ([AGE_BOUNDS] * len(AGE_CATEGORIES) +
          [DIS_BOUNDS] * len(DIS_CATEGORIES) +
          [GEO_BOUNDS] * len(REGIONS))

input_dict = {category: bound for category, bound in zip(categories, bounds)}
print(input_dict)

{'senior': (40, 240), 'child': (40, 240), 'dis_base': (0, 200), 'NORTH_EAST': (-50, 50), 'NORTH_WEST': (-50, 50), 'YORKSHIRE': (-50, 50), 'EAST_MIDLANDS': (-50, 50), 'WEST_MIDLANDS': (-50, 50), 'EAST_OF_ENGLAND': (-50, 50), 'LONDON': (-50, 50), 'SOUTH_EAST': (-50, 50), 'SOUTH_WEST': (-50, 50), 'WALES': (-50, 50), 'SCOTLAND': (-50, 50), 'NORTHERN_IRELAND': (-50, 50)}


In [3]:
def opt(reform):
    return optimize(input_dict, "mean_pct_loss", reform,
                    verbose=False, seed=0,
                    # Reforms don't always improve upon one another with the
                    # default tolerance of 0.01.
                    tol=0.0001)

In [4]:
%%time
reform_1 = opt("reform_1")
print(reform_1[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.514000e-01
losses                1.401361e+11
mean_pct_loss         5.190000e-02
mean_pct_loss_pwd2    5.850000e-02
gini                  2.749000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0519 

Optimal solution:
 adult               124
senior              205
child                76
dis_base              0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.05193292344448038
 message: 'Optimization terminated successfully.'
    nfev: 3182
     nit: 13
 success: True
       x: array([205.13710843,  75.56780549,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.     

In [5]:
%%time
reform_2 = opt("reform_2")
print(reform_2[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.560000e-01
losses                1.421457e+11
mean_pct_loss         4.970000e-02
mean_pct_loss_pwd2    5.130000e-02
gini                  2.706000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0497 

Optimal solution:
 adult               119
senior              197
child                77
dis_base             68
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.049682607946630826
 message: 'Optimization terminated successfully.'
    nfev: 3857
     nit: 16
 success: True
       x: array([196.57570623,  76.65245802,  68.32625414,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.    

In [6]:
%%time
reform_3 = opt("reform_3")
print(reform_3[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.576000e-01
losses                1.415572e+11
mean_pct_loss         4.960000e-02
mean_pct_loss_pwd2    5.130000e-02
gini                  2.709000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0496 

Optimal solution:
 adult               115
senior              193
child                72
dis_base             67
NORTH_EAST            7
NORTH_WEST            7
YORKSHIRE             0
EAST_MIDLANDS         3
WEST_MIDLANDS         3
EAST_OF_ENGLAND       2
LONDON                7
SOUTH_EAST            4
SOUTH_WEST            6
WALES                 3
SCOTLAND              4
NORTHERN_IRELAND      2
dtype: int64
     fun: 0.04961487560057053
 message: 'Optimization terminated successfully.'
    nfev: 15557
     nit: 68
 success: True
       x: array([195.13867201,  74.68219624,  67.03021849,   5.00022271,
         4.22973545,  -2.31622481,   0.76223605,   1.11094293,
        -0.23237491,   4.57357959,   1.39553331,   3.32186789,
         0.3519

Check that iterations improve.

In [7]:
assert reform_2[0].fun < reform_1[0].fun, "Reform 2 doesn't improve on 1"
assert reform_3[0].fun < reform_2[0].fun, "Reform 3 doesn't improve on 2"

Construct DataFrame to export, with parameter and optimization values.

In [8]:
ubi_params = pd.DataFrame(reform_1[1]).T
ubi_params.loc[1] = reform_2[1]
ubi_params.loc[2] = reform_3[1]
ubi_params['mean_pct_loss'] = [reform_1[0].fun,
                               reform_2[0].fun,
                               reform_3[0].fun]
ubi_params

,adult,senior,child,dis_base,NORTH_EAST,NORTH_WEST,YORKSHIRE,EAST_MIDLANDS,WEST_MIDLANDS,EAST_OF_ENGLAND,LONDON,SOUTH_EAST,SOUTH_WEST,WALES,SCOTLAND,NORTHERN_IRELAND,mean_pct_loss
0,123.956949,205.137108,75.567805,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.051933
1,119.002336,196.575706,76.652458,68.326254,0.000000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.049683
2,114.853116,192.822447,72.365971,67.030218,7.316448,6.54596,0.0,3.078461,3.427168,2.08385,6.889804,3.711758,5.638093,2.668136,4.31553,2.316225,0.049615


In [9]:
ubi_params.to_csv("optimal_params.csv", index=False)